In [ ]:
import pandas as pd
import requests

In [ ]:
URL_LYON = "https://download.data.grandlyon.com/wfs/rdata?SERVICE=WFS&VERSION=1.1.0&outputformat=GEOJSON&request=GetFeature&typename=jcd_jcdecaux.jcdvelov&SRSNAME=urn:ogc:def:crs:EPSG::4171"
URL_RENNES = "https://data.rennesmetropole.fr/api/records/1.0/search/?dataset=etat-des-stations-le-velo-star-en-temps-reel"

In [ ]:
def requestAPI(url):
  req = requests.get(url)
  return req.json()

In [ ]:
req_lyon = requestAPI(URL_LYON)

In [ ]:
req_rennes = requestAPI(URL_RENNES)

In [ ]:
def recup_datas_lyon(req_lyon):
  features = req_lyon["features"]

  stations = []

  for station in features:
    new_station = {}

    properties = station["properties"]

    new_station["nom"] = properties["name"]
    new_station["id_station"] = properties["number"]
    new_station["nb_emplacements"] = properties["bike_stands"]
    new_station["nb_emplacements_dispo"] = properties["available_bike_stands"]
    new_station["nb_velos_dispo"] = properties["available_bikes"]
    new_station["etat"] = properties["status"]
    new_station["latitude"] = properties["lat"]
    new_station["longitude"] = properties["lng"]
    new_station["derniere_maj"] = properties["last_update"]

    stations.append(new_station)

  return stations

In [ ]:
def recup_datas_rennes(req_rennes):
  records = req_rennes["records"]

  stations = []

  for station in records:
    new_station = {}

    fields = station["fields"]
    
    new_station["nom"] = fields["nom"]
    new_station["id_station"] = fields["idstation"]
    new_station["nb_emplacements"] = fields["nombreemplacementsactuels"]
    new_station["nb_emplacements_dispo"] = fields["nombreemplacementsdisponibles"]
    new_station["nb_velos_dispo"] = fields["nombrevelosdisponibles"]
    new_station["etat"] = fields["etat"]
    new_station["latitude"] = fields["coordonnees"][0]
    new_station["longitude"] = fields["coordonnees"][1]
    new_station["derniere_maj"] = fields["lastupdate"] 

    stations.append(new_station)

  return stations

In [ ]:
rennes_stations = pd.DataFrame(recup_datas_rennes(req_rennes))
rennes_stations

,nom,id_station,nb_emplacements,nb_emplacements_dispo,nb_velos_dispo,etat,latitude,longitude,derniere_maj
0,Musée Beaux-Arts,5510,16,3,13,En fonctionnement,48.109601,-1.674080,2022-03-23T11:35:06+00:00
1,Plaine de Baud,5546,24,9,15,En fonctionnement,48.112461,-1.643674,2022-03-23T11:35:06+00:00
2,Brest - Verdun,5534,24,14,10,En fonctionnement,48.113009,-1.693264,2022-03-23T11:35:06+00:00
3,Marbeuf,5538,20,10,10,En fonctionnement,48.111749,-1.702077,2022-03-23T11:35:06+00:00
4,Gros-Chêne,5578,18,12,6,En fonctionnement,48.125408,-1.664806,2022-03-23T11:35:06+00:00
5,Painlevé,5580,29,22,7,En fonctionnement,48.123506,-1.659762,2022-03-23T11:35:06+00:00
6,Joliot-Curie - Chateaubriand,5581,30,30,0,En fonctionnement,48.124389,-1.651203,2022-03-23T11:35:06+00:00
7,Pont de Strasbourg,5587,20,5,15,En fonctionnement,48.109859,-1.656028,2022-03-23T11:35:06+00:00
8,Mabilais,5591,25,11,14,En fonctionnement,48.105086,-1.693069,2022-03-23T11:35:06+00:00
9,Champs Libres,5516,24,17,7,En fonctionnement,48.105537,-1.674328,2022-03-23T11:35:06+00:00


In [ ]:
rennes_stations.nunique()

nom                      10
id_station               10
nb_emplacements           7
nb_emplacements_dispo     9
nb_velos_dispo            8
etat                      1
latitude                 10
longitude                10
derniere_maj              1
dtype: int64

In [ ]:
rennes_stations.isna().sum()

nom                      0
id_station               0
nb_emplacements          0
nb_emplacements_dispo    0
nb_velos_dispo           0
etat                     0
latitude                 0
longitude                0
derniere_maj             0
dtype: int64

In [ ]:
lyon_stations = pd.DataFrame(recup_datas_lyon(req_lyon))
lyon_stations

,nom,id_station,nb_emplacements,nb_emplacements_dispo,nb_velos_dispo,etat,latitude,longitude,derniere_maj
0,Place Mendès France,8010,20,18,2,OPEN,45.7400516465032750,4.8594164128821960,2022-03-23 12:28:43
1,Servant / Jaboulay,7055,16,0,16,OPEN,45.7467662258473740,4.8420506993495060,2022-03-23 12:31:50
2,Saint Fons - 4 chemins,17002,11,0,11,OPEN,45.7084437290321550,4.8548610775585280,2022-03-23 12:28:49
3,Place des Compagnons de la chanson,5002,18,12,6,OPEN,45.7565273368784000,4.7966121788894250,2022-03-23 12:25:12
4,Place Ampère,2003,10,0,10,OPEN,45.7530445329675700,4.8282455395393775,2022-03-23 12:30:15
...,...,...,...,...,...,...,...,...,...
426,Martinière / Monplaisir,8054,20,12,8,OPEN,45.7408946957303000,4.8683054018251575,2022-03-23 12:27:33
427,Lacour / Artillerie,7025,20,0,19,OPEN,45.7294551265970500,4.8441778720101850,2022-03-23 12:31:36
428,Créqui / Voltaire,3018,30,26,4,OPEN,45.7566983441751900,4.8484034857557150,2022-03-23 12:33:40
429,Gailleton,2012,16,1,14,OPEN,45.7534495026210240,4.8333645518023690,2022-03-23 12:28:01


In [ ]:
lyon_stations.nunique()

nom                      428
id_station               431
nb_emplacements           35
nb_emplacements_dispo     38
nb_velos_dispo            37
etat                       3
latitude                 431
longitude                431
derniere_maj             287
dtype: int64

In [ ]:
lyon_stations.isna().sum()

nom                      0
id_station               0
nb_emplacements          0
nb_emplacements_dispo    0
nb_velos_dispo           0
etat                     0
latitude                 0
longitude                0
derniere_maj             0
dtype: int64

In [ ]:
WHETHER_API_KEY = "3dfeff6f304e5faa5bbb3f4598a260e3"

In [ ]:
whether_lyon_req = requestAPI(f"https://api.openweathermap.org/data/2.5/weather?q=Lyon&units=metric&lang=fr&appid={WHETHER_API_KEY}")
whether_rennes_req = requestAPI(f"https://api.openweathermap.org/data/2.5/weather?q=Rennes&units=metric&lang=fr&appid={WHETHER_API_KEY}")

In [ ]:
def recup_infos_whether(whether_json):
  whether_informations = {}

  main = whether_json["main"]
  whether_informations["Température"] = main["temp"]
  whether_informations["Température ressentie"] = main["feels_like"]

  whether = whether_json["weather"][0]
  whether_informations["Temps"] = whether["description"]

  return whether_informations

In [ ]:
recup_infos_whether(whether_lyon_req)

{'Temps': 'ciel dégagé', 'Température': 13.65, 'Température ressentie': 11.93}

In [ ]:
recup_infos_whether(whether_rennes_req)

{'Temps': 'ciel dégagé', 'Température': 15.17, 'Température ressentie': 13.73}